# MVC project: Get the Data
- [GitHub](https://github.com/romainmartinez/mvc)

## 0. Setup

In [1]:
# Common imports
import pandas as pd
import numpy as np
import os

# the 'mvc' directory contains functions used but not necessary to understand the story
import mvc

# Figures
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
sns.set_context("paper")

# to make this notebook's output stable across runs
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

## 1. Load data

In [2]:
df = pd.read_hdf('mvc.h5')
df.head()

test,muscle,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,max,best_test
0,0,43,62,57,78,100,23,13,11,0,0,0,0,0,0,0,0,100,4
1,1,8,100,96,31,46,34,30,12,0,0,0,0,0,0,0,0,100,1
2,2,8,68,100,56,63,23,46,5,0,0,0,0,0,0,0,0,100,2
3,3,5,23,39,100,82,2,3,11,0,0,0,0,0,0,0,0,100,3
4,4,16,85,100,50,71,20,5,6,0,0,0,0,0,0,0,0,100,2


## 2. Explore

In [3]:
import tensorflow as tf
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt

/home/romain/miniconda3/envs/mvc/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [4]:
X = np.array(df.drop(['muscle', 'max', 'best_test'], axis=1)).astype('float64')
Y = np.array(df['max']).astype('float64')

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED,
                                       stratify=np.array(df['muscle']))

In [5]:
w = tf.Variable(tf.truncated_normal([16, 1], mean=0.0, stddev=1.0, dtype=tf.float64))
b = tf.Variable(tf.zeros(1, dtype = tf.float64))

In [6]:
def calc(x, y):
    # Returns predictions and error
    predictions = tf.add(b, tf.matmul(x, w))
    error = tf.reduce_mean(tf.square(y - predictions))
    return [ predictions, error ]

In [9]:
y, cost = calc(X_train, y_train)
# Feel free to tweak these 2 values:
learning_rate = 0.025
epochs = 3000
points = [[], []] # You'll see later why I need this

In [10]:
init = tf.global_variables_initializer()
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [15]:
import keras
from keras.layers import Input, Dense
from keras.models import Model

inputs = Input(shape=(1,))
preds = Dense(1,activation='linear')(inputs)

model = Model(inputs=inputs,outputs=preds)
sgd=keras.optimizers.SGD()
model.compile(optimizer=sgd ,loss='mse',metrics=['mse'])
model.fit(x,y, batch_size=1, epochs=30, shuffle=False)

NameError: name 'x' is not defined